# Lab 2: RESP Protocol Understanding

**Duration:** 45 minutes  
**Objective:** Understand Redis Serialization Protocol (RESP) using Python

## 🎯 Learning Objectives

- Understand RESP protocol format and data types
- Monitor Redis commands at protocol level
- Implement custom RESP parsing in Python
- Debug Redis communication
- Analyze protocol-level performance

## Part 1: Setup and RESP Basics (10 minutes)

In [ ]:
import redis
import socket
import time
import threading
from colorama import init, Fore, Style
import struct

init(autoreset=True)

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Test connection
if r.ping():
    print(f"{Fore.GREEN}✅ Connected to Redis{Style.RESET_ALL}")
else:
    print(f"{Fore.RED}❌ Failed to connect{Style.RESET_ALL}")

### Understanding RESP Data Types

In [ ]:
class RESPParser:
    """Custom RESP protocol parser for educational purposes."""
    
    @staticmethod
    def encode_command(command, *args):
        """Encode a Redis command in RESP format."""
        parts = [command] + list(args)
        resp = f"*{len(parts)}\r\n"
        
        for part in parts:
            part_str = str(part)
            resp += f"${len(part_str)}\r\n{part_str}\r\n"
        
        return resp.encode('utf-8')
    
    @staticmethod
    def decode_response(data):
        """Decode a RESP response."""
        if not data:
            return None
            
        data_str = data.decode('utf-8') if isinstance(data, bytes) else data
        
        # Simple string
        if data_str[0] == '+':
            return data_str[1:].strip()
        
        # Error
        elif data_str[0] == '-':
            return f"ERROR: {data_str[1:].strip()}"
        
        # Integer
        elif data_str[0] == ':':
            return int(data_str[1:].strip())
        
        # Bulk string
        elif data_str[0] == '$':
            length = int(data_str[1:data_str.index('\r\n')])
            if length == -1:
                return None
            start = data_str.index('\r\n') + 2
            return data_str[start:start + length]
        
        # Array
        elif data_str[0] == '*':
            count = int(data_str[1:data_str.index('\r\n')])
            return f"Array with {count} elements"
        
        return data_str

# Test RESP encoding
parser = RESPParser()

print(f"{Fore.CYAN}=== RESP PROTOCOL EXAMPLES ==={Style.RESET_ALL}\n")

# Example 1: SET command
set_cmd = parser.encode_command('SET', 'user:1001', 'John Doe')
print("SET Command in RESP:")
print(set_cmd.decode('utf-8').replace('\r\n', '\\r\\n'))
print()

# Example 2: GET command
get_cmd = parser.encode_command('GET', 'user:1001')
print("GET Command in RESP:")
print(get_cmd.decode('utf-8').replace('\r\n', '\\r\\n'))

## Part 2: Protocol-Level Communication (15 minutes)

In [ ]:
class RedisProtocolClient:
    """Low-level Redis client using raw sockets."""
    
    def __init__(self, host='localhost', port=6379):
        self.host = host
        self.port = port
        self.socket = None
    
    def connect(self):
        """Establish socket connection to Redis."""
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect((self.host, self.port))
        return self
    
    def send_command(self, command, *args):
        """Send a command using RESP protocol."""
        resp_data = RESPParser.encode_command(command, *args)
        self.socket.send(resp_data)
        
        # Read response
        response = self.socket.recv(1024)
        return response
    
    def close(self):
        """Close socket connection."""
        if self.socket:
            self.socket.close()

# Test raw protocol communication
print(f"{Fore.CYAN}=== RAW PROTOCOL COMMUNICATION ==={Style.RESET_ALL}\n")

try:
    # Create low-level client
    protocol_client = RedisProtocolClient()
    protocol_client.connect()
    
    # Send PING command
    print("Sending PING command...")
    response = protocol_client.send_command('PING')
    print(f"Raw Response: {response}")
    print(f"Decoded: {RESPParser.decode_response(response)}\n")
    
    # Send SET command
    print("Sending SET command...")
    response = protocol_client.send_command('SET', 'protocol:test', 'Hello RESP')
    print(f"Raw Response: {response}")
    print(f"Decoded: {RESPParser.decode_response(response)}\n")
    
    # Send GET command
    print("Sending GET command...")
    response = protocol_client.send_command('GET', 'protocol:test')
    print(f"Raw Response: {response}")
    
    protocol_client.close()
    
except Exception as e:
    print(f"{Fore.RED}Error: {e}{Style.RESET_ALL}")

## Part 3: Monitoring Redis Commands (10 minutes)

In [ ]:
class RedisMonitor:
    """Monitor Redis commands in real-time."""
    
    def __init__(self, redis_client):
        self.client = redis_client
        self.monitoring = False
        self.commands = []
    
    def start_monitoring(self, duration=5):
        """Monitor Redis commands for specified duration."""
        print(f"{Fore.CYAN}=== MONITORING REDIS COMMANDS ==={Style.RESET_ALL}")
        print(f"Monitoring for {duration} seconds...\n")
        
        # Create monitoring connection
        monitor_conn = redis.Redis(host='localhost', port=6379, decode_responses=True)
        pubsub = monitor_conn.pubsub()
        
        # Track commands
        start_time = time.time()
        command_count = 0
        
        # Simulate some operations
        operations = [
            ('SET', 'monitor:key1', 'value1'),
            ('GET', 'monitor:key1'),
            ('INCR', 'monitor:counter'),
            ('HSET', 'monitor:hash', 'field1', 'value1'),
            ('HGET', 'monitor:hash', 'field1'),
            ('DEL', 'monitor:key1')
        ]
        
        for op in operations:
            command = op[0]
            args = op[1:]
            
            # Execute command
            if command == 'SET':
                self.client.set(args[0], args[1])
            elif command == 'GET':
                self.client.get(args[0])
            elif command == 'INCR':
                self.client.incr(args[0])
            elif command == 'HSET':
                self.client.hset(args[0], args[1], args[2])
            elif command == 'HGET':
                self.client.hget(args[0], args[1])
            elif command == 'DEL':
                self.client.delete(args[0])
            
            # Log command
            timestamp = time.time() - start_time
            print(f"[{timestamp:.3f}s] {command} {' '.join(map(str, args))}")
            command_count += 1
            time.sleep(0.5)
        
        print(f"\n📊 Monitoring Summary:")
        print(f"  Total commands: {command_count}")
        print(f"  Commands/sec: {command_count/duration:.2f}")

# Run monitoring
monitor = RedisMonitor(r)
monitor.start_monitoring(5)

## Part 4: Protocol Performance Analysis (10 minutes)

In [ ]:
import statistics

def analyze_protocol_performance():
    """Analyze protocol-level performance metrics."""
    print(f"{Fore.CYAN}=== PROTOCOL PERFORMANCE ANALYSIS ==={Style.RESET_ALL}\n")
    
    # Test different data sizes
    test_sizes = [10, 100, 1000, 10000]
    results = []
    
    for size in test_sizes:
        data = 'x' * size
        times = []
        
        # Run multiple iterations
        for i in range(100):
            start = time.perf_counter()
            r.set(f'perf:test:{i}', data)
            r.get(f'perf:test:{i}')
            end = time.perf_counter()
            times.append((end - start) * 1000)  # Convert to ms
        
        # Calculate statistics
        avg_time = statistics.mean(times)
        min_time = min(times)
        max_time = max(times)
        
        results.append({
            'size': size,
            'avg_ms': avg_time,
            'min_ms': min_time,
            'max_ms': max_time
        })
        
        # Clean up
        for i in range(100):
            r.delete(f'perf:test:{i}')
    
    # Display results
    print("Data Size Impact on Performance:")
    print("-" * 60)
    print(f"{'Size (bytes)':<15} {'Avg (ms)':<15} {'Min (ms)':<15} {'Max (ms)':<15}")
    print("-" * 60)
    
    for result in results:
        print(f"{result['size']:<15} {result['avg_ms']:<15.3f} {result['min_ms']:<15.3f} {result['max_ms']:<15.3f}")
    
    print("\n💡 Insights:")
    print("  • Larger payloads increase latency")
    print("  • RESP protocol adds minimal overhead")
    print("  • Network latency dominates for small payloads")

# Run performance analysis
analyze_protocol_performance()